# Learning Tree Functions

In [136]:
def attr_domain(attribute, examples):
    return examples[attribute].drop_duplicates()


class Node:
    def __init__(self, attribute):
        self.attribute = attribute
        self.branch = {}


def plurality_value(examples):  # gests most common target value
    c = Counter(examples.target).most_common(2)
    if c[1][0] == c[1][1]:
        return Node(c[0][random.randint(0, 1)])
    else:
        return Node(c[0][0])


def importance(attributes, examples):
    def bool_entropy(q):  # B(q)
        if q == 0 or q == 1:
            return 0
        return -(q * math.log(q, 2) + (1 - q) * math.log(1 - q, 2))

    def expected_entropy(attribute, examples):  # Remainder(A)
        sum = 0.0
        for k in attr_domain(attribute, examples):
            ek = examples[examples[attribute] == k]
            pk = len(ek[ek['target'] == ' >50K'])
            nk = len(ek[ek['target'] == ' <=50K'])
            q = float(pk) / float(pk + nk)
            sum += float(len(ek)) / len(examples) * bool_entropy(q)
        return sum

    gain_list = []  # list of Gain(A)
    for a in attributes:
        gain_list.append(
            bool_entropy(len(examples[examples['target'] == ' >50K']) / float(len(examples))) - expected_entropy(a,
                                                                                                                 examples))
    index, value = max(enumerate(gain_list), key=operator.itemgetter(1))
    return attributes[index]


def decision_tree_learning(examples, attributes, parent_examples=()):
    if len(examples) == 0:
        return plurality_value(parent_examples)
    elif len(set(examples.target)) == 1:  # if same classification
        return Node(examples.iat[0, len(examples.columns) - 1])
    elif len(attributes) == 0:
        return plurality_value(examples)
    else:
        A = importance(attributes, examples)
        tree = Node(A)
        a = list(attributes)
        a.remove(A)
        for v in attr_domain(A, examples):
            exs = examples[examples[A] == v]
            tree.branch[v] = decision_tree_learning(exs, a, examples)
        return tree
    
     
def to_rules(tree):     # convert the tree to DNF rules
    def visit(node, path):
        if len(node.branch) != 0:
            for branch in node.branch:
                path.append([node.attribute, branch])
                visit(node.branch[branch], path)
                path.pop()
        else:
            p = list(path)
            p.append(['target', node.attribute])
            rules.append(p)
    rules = []
    path = []
    visit(tree, path)
    return rules

# Preprocess Adult Dataset

In [137]:
%matplotlib notebook
import pandas as pd
import math
from collections import Counter
import random
import operator
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv('adult.data')

df.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'target']
df = df.drop(columns=['fnlwgt'])
df = df.drop(columns=['capital_gain'])
df = df.drop(columns=['capital_loss'])
df = df[df.workclass != ' ?']
df = df[df.occupation != ' ?']
df = df[df.native_country != ' ?']
df.loc[df['native_country'] != ' United-States', 'native_country'] = 'Foreign-Country'
df['age'] =  (pd.cut(df['age'], bins=5))
df['hours_per_week'] = (pd.cut(df['hours_per_week'], bins=5))
attributes = df.columns.tolist()
attributes.remove('target')
training_set, test_set = train_test_split(df, test_size=0.2)
training_set, validation_set = train_test_split(df, test_size=0.33)

# Pruning

In [138]:
def rules_pruning(limit):
    def rule_lengths():
        l = []
        for rule in rules:
            l.append(len(rule))
        return l

    def errors(rule):
        vs = validation_set
        if rule is None or len(rule) <= 2:
            return 100000
        for r in rule[:-1]:
            vs = vs[vs[r[0]] == r[1]]
        r = rule[-1]
        e = len(vs[vs['target'] != r[1]])
        return e

    def score(rule):
        if len(rule) == 0:
            return 0
        vs = validation_set
        for r in rule[:-1]:
            vs = vs[vs[r[0]] == r[1]]
        total = len(vs)
        r = rule[-1]
        correct = len(vs[vs['target'] == r[1]])
        if total == 0:
            return 0
        return float(correct) / total

    def best_r(original_rule):  # sceglie la miglior r da togliere
        if len(original_rule) == 0:
            return False
        scores = []
        for r in original_rule[:-1]:
            rule = list(original_rule)
            rule.remove(r)
            scores.append(score(rule))
        return original_rule[(scores.index(max(scores)))]

    def change_rule_target(rule):
        if len(rule) == 0:
            return 0
        vs = validation_set
        for r in rule[:-1]:
            vs = vs[vs[r[0]] == r[1]]
        r = rule[-1]
        if len(vs) > 0:
            r[1] = Counter(vs.target).most_common()[0][0]
        return 0

    def is_in(smaller, bigger):  # Note: does not check last value
        if len(smaller) == 0 or len(bigger) == 0:
            return False
        for s in smaller[:-1]:
            match = False
            for b in bigger[:-1]:
                if s == b:
                    match = True
                    break
            if not match:
                return False
        return True

    def pruning(index):  # pruning of rules[index]
        if len(rules[index]) <= 2:
            return False
        original_rule = list(rules[index])
        r = best_r(rules[index])
        rules[index].remove(r)
        change_rule_target(rules[index])
        err_pruned = errors(rules[index])

        err_non_pruned = 0
        for rule in rules:
            if rules[index] != rule and is_in(rules[index], rule):
                err_non_pruned += errors(rule)
        err_non_pruned += errors(original_rule)
        if err_pruned >= err_non_pruned:  # turn back
            rules[index] = original_rule
            return False
        else:
            return True


    lengths = rule_lengths()
    while sum(lengths) != 0:
        max_l = max(lengths)
        if max_l <= limit:
            break
        for i in range(len(rules)):
            if lengths[i] == max_l:  # chooses only rules with length max_l
                p = pruning(i)
                if p:  # cleanup
                    lengths[i] -= 1
                    for l in range(len(rules)):
                        if is_in(rules[i], rules[l]) and i != l:
                            lengths[l] = 0
                            rules[l] = []
                else:
                    lengths[i] = 0
    for i in reversed(range(len(rules))):
        if len(rules[i]) == 0:
            rules.pop(i)


In [139]:
dtree = decision_tree_learning(training_set, attributes)
rules = to_rules(dtree)
rules_pruning(4)